In [6]:
import itertools
import numpy as np
import functools

In [7]:
%display latex

In [8]:
def same(*args):
    for arg in args:
        assert args[0] == arg
    return args[0]

In [4]:
class GaussQuad:
    def __init__(self, alphas, betas, wf):
        self.n = same(len(alphas), len(betas))
        
        self.wf = wf
    
        J = np.zeros((self.n, self.n))
        for k, alpha, beta in zip(range(self.n), alphas, betas):
            J[k, k] = alpha
            if k-1 >= 0:
                J[k-1, k] = sqrt(beta)
                J[k, k-1] = sqrt(beta)

        x = var("x")
        polys = [0, 1]
        for k, alpha, beta in zip(itertools.count(2), alphas, betas):
            poly = (x - alpha)*polys[k-1] - beta*polys[k-2]
            polys.append(poly)
        self.poly = polys[-1]
        self.poly = self.poly.full_simplify()
        
        # Eigenvectors are returned in normalized form.
        # eig.eigenvectors[vector_dimension,vector]
        eig = np.linalg.eig(J)
        eig_val = eig.eigenvalues
        eig_vec = eig.eigenvectors.T

        self.x = eig_val.tolist()
        v = eig_vec.tolist()

        self.w = []
        for k in range(self.n):
            self.w.append(betas[0]*v[k][0]**2)
    
    @property
    def sx(self):
        return var("x")
    
    @property
    def sf(self):
        return function("f")
    
    @functools.cached_property
    def quad(self):
        quad = 0
        for xv, wv in zip(self.x, self.w):
            quad += wv*(self.sf)(xv)
        return quad
    
    @functools.cached_property
    def poly_norm(self):
        coef = max(self.poly.coefficients(), key=lambda a: a[1])[0]
        poly = self.poly / coef
        return poly
        
    @property
    def rest(self):
        a = var("a")
        b = var("b")
        
        with assuming(a < b):
            rest = (
                diff((self.sf)(self.sx), self.sx, 2*self.n)
                / factorial(2*self.n)
                * integral(self.wf(self.sx)*self.poly_norm**2, self.sx, a, b)
            )
        return rest

In [5]:
def laguerre_values(n, alpha):
    alphas = []
    betas = []
    
    for k in range(n):
        alphas.append(2*k+alpha+1)
        if k == 0:
            betas.append(gamma(1+alpha))
        else:
            betas.append(k*(k+alpha))
    
    w = function("w")
    x = var("x")
    w(x) = x**alpha * e**(-x)
    
    return alphas, betas, w

In [8]:
gq = GaussQuad(*laguerre_values(4, 0))
gq.quad

0.0005392947055613282*f(9.395070912301133) + 0.038887908515005364*f(4.53662029692113) + 0.35741869243779917*f(1.7457611011583472) + 0.6031541043416337*f(0.3225476896193926)

# Incercare

In [10]:
a, b = var("a b")
x = var("x")
f = function("f")

birk = (x-b) * diff(f(x), x)(x=a) + f(b)
display(birk)

birki = integral(birk, x, a, b)
display(birki)

-(b - x)*diff(f(a), a) + f(b)

-1/2*a^2*diff(f(a), a) + a*b*diff(f(a), a) - 1/2*b^2*diff(f(a), a) - a*f(b) + b*f(b)

# 1

In [9]:
f = function("f")
t = var("t")
t3, t4 = var("t_3 t_4")
a0, a1, a2, a3 = var("a_0 a_1 a_2 a_3")

poly = a0*f(0) + a1*diff(f(t), t)(t=0) + a2*f(t3) + a3*f(t4)

display(poly)

sol = solve([
    integral(e**-t*t**0, t, 0, oo) == poly.substitute_function(f, (t**0).function(t)),
    integral(e**-t*t**1, t, 0, oo) == poly.substitute_function(f, (t**1).function(t)),
    integral(e**-t*t**2, t, 0, oo) == poly.substitute_function(f, (t**2).function(t)),
    integral(e**-t*t**3, t, 0, oo) == poly.substitute_function(f, (t**3).function(t)),
    integral(e**-t*t**4, t, 0, oo) == poly.substitute_function(f, (t**4).function(t)),
    integral(e**-t*t**5, t, 0, oo) == poly.substitute_function(f, (t**5).function(t)),
], [a0, a1, a2, a3, t3, t4])
# display(sol)

polys = poly.substitute(sol[0]).function(t)
display(polys)

a_0*f(0) + a_2*f(t_3) + a_3*f(t_4) + a_1*D[0](f)(0)

t |--> 1/72*f(6) + 3/8*f(2) + 11/18*f(0) + 1/6*D[0](f)(0)

In [10]:
R = (f(t) - polys(t)).function(t)

In [11]:
R

t |--> -1/72*f(6) - 3/8*f(2) - 11/18*f(0) + f(t) - 1/6*D[0](f)(0)

In [4]:
display(R.substitute_function(f, (t).function(t)))

t - 1

In [15]:
pf = (t**0).function(t)
display(R.substitute_function(f, pf))

pf = (t**1).function(t)
display(R.substitute_function(f, pf))

# pf = (t**2).function(t)
# display(R.substitute_function(f, pf))

0

t - 1

In [21]:
f = function("f")
t = var("t")
t3, t4 = var("t_3 t_4")
A1, A2, A3, A4 = var("A_1 A_2 A_3 A_4")

int_ = integral(f(t), t, -1, 1)
poly = (A1*f(-1) + A2*diff(f(t), t)(t=-1) + A3*f(t3) + A4*f(t4)).function(t, t3, t4)

display(int_)
display(poly)

sol = solve([
    integral(1, t, -1, 1) == f(t, 1, 1),
    integral(t, t, -1, 1) == f(t, t3, t4),
    integral(t**2, t, -1, 1) == f(t, t3**2, t4**2),
    integral(t**3, t, -1, 1) == f(t, t3**3, t4**3),
], [A1, A2, A3, A4])

# sol = solve(int_ == poly, [A1, A2, A3, A4])
display(sol)

integrate(f(t), t, -1, 1)

(t, t_3, t_4) |--> A_1*f(-1) + A_3*f(t_3) + A_4*f(t_4) + A_2*D[0](f)(-1)

[]

# 2

## (a)

$$
f(\alpha)=0
$$

In [142]:
f = function("f")
g = function("g")
x = var("x")
alpha = var("alpha")

g(x) = x - f(x) / sqrt(diff(f(x), x)(x=x)**2 - f(x)*diff(f(x), x, 2)(x=x))
display(g)

display(g(x=alpha).substitute(f(alpha) == 0))

x |--> x - f(x)/sqrt(diff(f(x), x)^2 - f(x)*diff(f(x), x, x))

alpha

In [143]:
gd = diff(g(x), x)
display(gd(x=alpha).substitute(f(alpha) == 0).full_simplify())

(sqrt(diff(f(alpha), alpha)^2) - diff(f(alpha), alpha))/sqrt(diff(f(alpha), alpha)^2)

Daca ca $f'(\alpha)\gt0$, atunci vom avea $g'(\alpha)=0$. Altfel, $g'(\alpha)=2$.

In [144]:
gdd = diff(gd, x)
gdd(x=alpha).substitute(f(alpha) == 0).full_simplify()

0

Asadar $g''(\alpha)=0$.

In [145]:
gddd = diff(gdd, x)
gddd(x=alpha).substitute(f(alpha) == 0).full_simplify()
# solve(gddd(x=alpha).substitute(f(alpha) == 0) == 0, [f(alpha)])

1/4*(3*diff(f(alpha), alpha, alpha)^2 - 4*diff(f(alpha), alpha)*diff(f(alpha), alpha, alpha, alpha))/(sqrt(diff(f(alpha), alpha)^2)*diff(f(alpha), alpha))

Deoarece $g'(\alpha)=g''(\alpha)=0$, inseamna ca gradul de convergenta al sirului este cel putin $p=3$, pentru $f'(\alpha)\gt0$ (teorema). Asadar, sirul generat converge cubic.

# 4

## (a)

In [211]:
f = function("f")
x = var("x")
alpha = var("alpha")

g = (
    x - 2*f(x)
    / (
        diff(f(x), x)(x=x)
        * (1 + sqrt(1 - 2*f(x)*diff(f(x), x, 2)(x=x)/diff(f(x), x)(x=x)**2))
    )
).function(x)
display(g)

x |--> x - 2*f(x)/((sqrt(-2*f(x)*diff(f(x), x, x)/diff(f(x), x)^2 + 1) + 1)*diff(f(x), x))

In [212]:
display(g(alpha).substitute(f(alpha) == 0))

alpha

In [213]:
gd = diff(g(x), x)
gd(x=alpha).substitute(f(alpha) == 0)

0

Asadar $g'(\alpha)=0$.

In [214]:
gdd = diff(gd, x)
gdd(x=alpha).substitute(f(alpha) == 0)

0

Asadar $g''(\alpha)=0$.

In [215]:
gddd = diff(gdd, x)
gddd(x=alpha).substitute(f(alpha) == 0).full_simplify()

-diff(f(alpha), alpha, alpha, alpha)/diff(f(alpha), alpha)

Deoarece $g'(\alpha)=g''(\alpha)=0$, inseama ca gradul de convergenta al sirului este cel putin $p=3$ (teorema). Asadar, sirul generat converge cubic.

# 3

In [183]:
poly.substitute_function(f, (t**2).function(t))

A_2*t_3^2 + A_3*t_4^2 + A_0 - 2*A_1

In [219]:
poly

A_0*f(-1) + A_1*D[0](f)(-1)

In [237]:
poly.substitute_function(f, (t**1).function(t))

A_0 + A_1

In [238]:
poly.substitute_function(f, (t**0).function(t))

A_0

In [1]:
f = function("f")
t = var("t")
t3, t4 = var("t_3 t_4")
A0, A1, A2, A3 = var("A_0 A_1 A_2 A_3")

# poly = A0*f(-1) + A1*diff(f(t), t)(t=-1) + A2*f(t3) + A3*f(t4)
poly = A0*f(1) + A1*diff(f(t), t)(t=-1)
display(poly)

# poly = A2*f(t3) + A3*f(t4)
R = (f(t) - poly).function(t)
display(R)

sol = solve([
    integral(t**0, t, -1, 1) - poly.substitute_function(f, (t**0).function(t)) == 0,
    integral(t**1, t, -1, 1) - poly.substitute_function(f, (t**1).function(t)) == 0,
#     integral(t**2, t, -1, 1) == poly.substitute_function(f, (t**2).function(t)),
#     integral(t**3, t, -1, 1) == poly.substitute_function(f, (t**3).function(t)),
#     integral(t**4, t, -1, 1) == poly.substitute_function(f, (t**4).function(t)),
#     integral(t**5, t, -1, 1) == poly.substitute_function(f, (t**5).function(t)),
#     R.substitute_function(f, (t**0).function(t)) == 0,
#     R.substitute_function(f, (t**1).function(t)) == 0,
#     A2==0,
#     A3==0,
#     R.substitute_function(f, (t**2).function(t)) == 0,
#     R.substitute_function(f, (t**3).function(t)) == 0,
#     R.substitute_function(f, (t**4).function(t)) == 0,
#     R.substitute_function(f, (t**5).function(t)) == 0,
# ], [A0, A1, A2, A3, t3, t4])
], [A0, A1])
display(sol)

# polys = poly.substitute(sol[0]).function(t)
# display(polys)

A_0*f(1) + A_1*D[0](f)(-1)

t |--> -A_0*f(1) - A_1*D[0](f)(-1) + f(t)

[[A_0 == 2, A_1 == -2]]

In [2]:
poly.substitute(sol[0])

2*f(1) - 2*D[0](f)(-1)

In [3]:
Rs = R.substitute(sol[0])
display(Rs)

for p in range(7):
    pf = (t**p).function(t)
    display(Rs.substitute_function(f, pf))

t |--> -2*f(1) + f(t) + 2*D[0](f)(-1)

-1

t

t^2 - 6

t^3 + 4

t^4 - 10

t^5 + 8

t^6 - 14

# Incercare

Cautam polinomul Birkhoff care interpoleaza x0=-1 cu f(x0) si f'(x0), x1=t3 cu f(x1), x2=t4 cu f(x2)

n = 2 + 1 + 1 - 1 = 4

In [63]:
f = function("f")
x = var("x")
x1, x2 = var("x_1 x_2")
a0, a1, a2, a3 = var("a_0 a_1 a_2 a_3")

poly = (a0 + a1*x + a2*x**2 + a3*x**3).function(x)
display(poly)

sol = solve([
    poly(-1) == f(-1),
    diff(poly, x)(-1) == diff(f(x), x)(x=-1),
    poly(x1) == f(x1),
    poly(x2) == f(x2),
#     poly.substitute_function(f, (x**0).function(x)) == 1,
#     poly.substitute_function(f, (x**1).function(x)) == x,
], [a0, a1, a2, a3])
# display(sol)

polys = poly.substitute(sol[0])
display(polys)

x |--> a_3*x^3 + a_2*x^2 + a_1*x + a_0

x |--> -(x_1^2*f(x_2) - (x_1^2 - x_2^2 + 2*x_1 - 2*x_2)*f(-1) - (x_2^2 + 2*x_2 + 1)*f(x_1) + 2*x_1*f(x_2) - (x_1^2*(x_2 + 1) - (x_2^2 - 1)*x_1 - x_2^2 - x_2)*D[0](f)(-1) + f(x_2))*x^3/((x_2^2 + 2*x_2 + 1)*x_1^3 - (x_2^3 - 3*x_2 - 2)*x_1^2 - x_2^3 - (2*x_2^3 + 3*x_2^2 - 1)*x_1 - 2*x_2^2 - x_2) + (x_1^3*f(x_2) - (x_1^3 - x_2^3 - 3*x_1 + 3*x_2)*f(-1) - (x_2^3 - 3*x_2 - 2)*f(x_1) - 3*x_1*f(x_2) - (x_1^3*(x_2 + 1) - x_2^3 - (x_2^3 + 1)*x_1 + x_2)*D[0](f)(-1) - 2*f(x_2))*x^2/((x_2^2 + 2*x_2 + 1)*x_1^3 - (x_2^3 - 3*x_2 - 2)*x_1^2 - x_2^3 - (2*x_2^3 + 3*x_2^2 - 1)*x_1 - 2*x_2^2 - x_2) + (2*x_1^3*f(x_2) + 3*x_1^2*f(x_2) - (2*x_1^3 - 2*x_2^3 + 3*x_1^2 - 3*x_2^2)*f(-1) - (2*x_2^3 + 3*x_2^2 - 1)*f(x_1) + ((x_2^2 - 1)*x_1^3 - (x_2^3 + 1)*x_1^2 + x_2^3 + x_2^2)*D[0](f)(-1) - f(x_2))*x/((x_2^2 + 2*x_2 + 1)*x_1^3 - (x_2^3 - 3*x_2 - 2)*x_1^2 - x_2^3 - (2*x_2^3 + 3*x_2^2 - 1)*x_1 - 2*x_2^2 - x_2) + (x_1^3*f(x_2) + 2*x_1^2*f(x_2) + ((x_2^2 + 2*x_2)*x_1^3 - (x_2^3 - 3*x_2)*x_1^2 - (2*x_2^3 + 3*x_2^2)*x_1)*f(-1) - (x_2^3 + 2*x_2^2 + x_2)*f(x_1) + x_1*f(x_2) + ((x_2^2 + x_2)*x_1^3 - (x_2^3 - x_2)*x_1^2 - (x_2^3 + x_2^2)*x_1)*D[0](f)(-1))/((x_2^2 + 2*x_2 + 1)*x_1^3 - (x_2^3 - 3*x_2 - 2)*x_1^2 - x_2^3 - (2*x_2^3 + 3*x_2^2 - 1)*x_1 - 2*x_2^2 - x_2)

In [59]:
quad = integral(polys, x, -1, 1).function(x)
display(quad)

x |--> 1/12*(2*(3*x_1^2*(2*f(-1) + 3*D[0](f)(-1)) + 4*x_1*(3*f(-1) + D[0](f)(-1)) - 8*f(-1) + 14*f(x_1) - 5*D[0](f)(-1))*x_2^3 + 2*x_1^3*(8*f(-1) + 5*D[0](f)(-1)) - 3*(2*x_1^3*(2*f(-1) + 3*D[0](f)(-1)) - x_1*(12*f(-1) + 5*D[0](f)(-1)) + 5*f(-1) - 13*f(x_1) + D[0](f)(-1))*x_2^2 + 3*x_1^2*(5*f(-1) + D[0](f)(-1)) - (8*x_1^3*(3*f(-1) + D[0](f)(-1)) + 3*x_1^2*(12*f(-1) + 5*D[0](f)(-1)) - 18*f(-1) + 6*f(x_1) - 7*D[0](f)(-1))*x_2 - x_1*(18*f(-1) + 7*D[0](f)(-1)) - (28*x_1^3 + 39*x_1^2 - 6*x_1 - 17)*f(x_2) - 17*f(x_1))/((x_1^2 + 2*x_1 + 1)*x_2^3 - x_1^3 - (x_1^3 - 3*x_1 - 2)*x_2^2 - 2*x_1^2 - (2*x_1^3 + 3*x_1^2 - 1)*x_2 - x_1) + 1/12*(2*(3*x_1^2*(2*f(-1) + D[0](f)(-1)) + 4*x_1*(3*f(-1) + D[0](f)(-1)) + 4*f(-1) + 2*f(x_1) + D[0](f)(-1))*x_2^3 - 2*x_1^3*(4*f(-1) + D[0](f)(-1)) - 3*(2*x_1^3*(2*f(-1) + D[0](f)(-1)) - 3*x_1*(4*f(-1) + D[0](f)(-1)) - 5*f(-1) - 3*f(x_1) - D[0](f)(-1))*x_2^2 - 3*x_1^2*(5*f(-1) + D[0](f)(-1)) - (8*x_1^3*(3*f(-1) + D[0](f)(-1)) + 9*x_1^2*(4*f(-1) + D[0](f)(-1)) - 6*f(-1) - 6*f(x_1) - D[0](f)(-1))*x_2 - x_1*(6*f(-1) + D[0](f)(-1)) - (4*x_1^3 + 9*x_1^2 + 6*x_1 + 1)*f(x_2) + f(x_1))/((x_1^2 + 2*x_1 + 1)*x_2^3 - x_1^3 - (x_1^3 - 3*x_1 - 2)*x_2^2 - 2*x_1^2 - (2*x_1^3 + 3*x_1^2 - 1)*x_2 - x_1)

In [60]:
R = (f(x) - quad).function(x)

In [61]:
pf = (x**0).function(x)
display(R.substitute_function(f, pf))

pf = (x**1).function(x)
display(R.substitute_function(f, pf))

pf = (x**2).function(x)
display(R.substitute_function(f, pf))

-1

x - 1/12*(6*(x_1^2 + 2*x_1 + 1)*x_2^3 - 6*x_1^3 - 6*(x_1^3 - 3*x_1 - 2)*x_2^2 - 12*x_1^2 - (28*x_1^3 + 39*x_1^2 - 6*x_1 - 17)*x_2 + (16*x_1^3 + 21*x_1^2 - 6*x_1 - 11)*x_2 - 6*x_1)/((x_1^2 + 2*x_1 + 1)*x_2^3 - x_1^3 - (x_1^3 - 3*x_1 - 2)*x_2^2 - 2*x_1^2 - (2*x_1^3 + 3*x_1^2 - 1)*x_2 - x_1) + 1/12*(6*(x_1^2 + 2*x_1 + 1)*x_2^3 - 6*x_1^3 - 6*(x_1^3 - 3*x_1 - 2)*x_2^2 - 12*x_1^2 - (16*x_1^3 + 27*x_1^2 + 6*x_1 - 5)*x_2 + (4*x_1^3 + 9*x_1^2 + 6*x_1 + 1)*x_2 - 6*x_1)/((x_1^2 + 2*x_1 + 1)*x_2^3 - x_1^3 - (x_1^3 - 3*x_1 - 2)*x_2^2 - 2*x_1^2 - (2*x_1^3 + 3*x_1^2 - 1)*x_2 - x_1)

x^2 - 1/12*(4*(x_1^2 + 2*x_1 + 1)*x_2^3 - 4*x_1^3 - (28*x_1^3 + 39*x_1^2 - 6*x_1 - 17)*x_2^2 + 3*(8*x_1^3 + 13*x_1^2 + 2*x_1 - 3)*x_2^2 - 8*x_1^2 - 4*(2*x_1^3 + 3*x_1^2 - 1)*x_2 - 4*x_1)/((x_1^2 + 2*x_1 + 1)*x_2^3 - x_1^3 - (x_1^3 - 3*x_1 - 2)*x_2^2 - 2*x_1^2 - (2*x_1^3 + 3*x_1^2 - 1)*x_2 - x_1) - 1/12*(4*(x_1^2 + 2*x_1 + 1)*x_2^3 - 4*x_1^3 - (4*x_1^3 + 9*x_1^2 + 6*x_1 + 1)*x_2^2 + 9*(x_1^2 + 2*x_1 + 1)*x_2^2 - 8*x_1^2 - 4*(2*x_1^3 + 3*x_1^2 - 1)*x_2 - 4*x_1)/((x_1^2 + 2*x_1 + 1)*x_2^3 - x_1^3 - (x_1^3 - 3*x_1 - 2)*x_2^2 - 2*x_1^2 - (2*x_1^3 + 3*x_1^2 - 1)*x_2 - x_1)